In [3]:
import os
import sys
sys.path.append('/home/ubuntu/project')
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import warnings
warnings.simplefilter("ignore", UserWarning)
import cv2
import torch
import numpy as np
import time

from PIL import Image, ImageDraw
from utils.get_utube import get_youtube
from utils.display import video_display
from detectors import DSFD

In [2]:

res = ['720p']

videos = get_youtube("https://www.youtube.com/watch?v=6RLLOEzdxsM", res)


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Net = DSFD(device='cuda')

[DSFD] loading with cuda
[DSFD] finished loading (3.3385 sec)


In [14]:
for j, r in enumerate(res):
    frames = videos[j]
    
    frames_tracked = []
    
    start = time.monotonic()
    
    for i, frame in enumerate(frames[1300:2000]):
        print(f'Tracking resolution {r} \r', end=' ')
        print('Tracking frame: {}'.format(i + 1), end='\r')

        # Detect faces
        boxes = Net.detect_faces(frame, conf_th=0.8,scales=[0.5, 1.0])

        # Draw faces
        frame_draw = Image.fromarray(frame.copy())
        draw = ImageDraw.Draw(frame_draw)
        if boxes is not None:
            for box in boxes:
                draw.rectangle(box[:-1].tolist(), outline=(255, 0, 0), width=6)

        # Add to frame list
        frames_tracked.append(frame_draw)
    
    print(f'{r} tracked in {time.monotonic() - start}')

    dim = frames_tracked[0].size

    # Use vp9 codec and webm format to show video in notebook
    # TODO : Why opencv doesn't support h264? -> how can we show video in format mp4
    fourcc = cv2.VideoWriter_fourcc(*'VP90')   
    video_tracked = cv2.VideoWriter(f'video_tracked_{r}.webm', fourcc, 20.0, dim)
    for frame in frames_tracked:
        video_tracked.write(cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR))
    video_tracked.release()

360p tracked in 59.92813611199381
480p tracked in 87.65584451099858
720p tracked in 149.39574370899936


In [4]:
video_display('video_tracked_720p.webm')